<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# MTBS Fire Modeling Demo

_by Jeremy Freeman (CarbonPlan), September 19, 2020_

This notebook loads munged, downsampled MTBS fire data and TerraClimate climate
data and fits a simple logistical regression model.

Before getting started you'll need to download this file:
https://storage.googleapis.com/carbonplan-data/processed/mtbs/conus/16000m/training.pkl.

A file with the same contents is available in parquet:
https://storage.googleapis.com/carbonplan-data/processed/mtbs/conus/16000m/training.parquet.


### Setup


In [1]:
from showit import image
import xarray as xr
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
%matplotlib inline

In [46]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load the data


We've already munged the data into two dictionaries: X (with our regressors) and
y (with our fire data). We're using the downsampled data for speed and
simplicity. At higher resolutions, similar approaches should work, and
additional techniques could possibly be included.


In [3]:
from carbonplan_forests import load, setup, fit, plot

In [4]:
setup.plotting()

In [1422]:
coarsen = 16
tlim = (1984, 2018)
data_vars = ['ppt', 'tavg']

In [1423]:
mask = load.nlcd(store='local', classes='all', year=2001)
climate = load.terraclim(store='local', tlim=tlim, coarsen=coarsen, data_vars=data_vars, mask=mask)

In [1424]:
groups = load.nftd(store='local', groups='all', coarsen=coarsen, mask=mask, area_threshold=1500)

In [1425]:
mtbs = load.mtbs(store='local', coarsen=coarsen, tlim=tlim)
mtbs['vlf'] = mtbs['vlf'] > 0

In [1426]:
plot.fire.monthly(mtbs.sel(time='2017'), 'vlf', clim=(0, 1))

alt.VConcatChart(...)

In [1427]:
x = mtbs['time']
y = mtbs['vlf'].mean(['x', 'y'])

plot.line(x=x, y=y, width=900, height=200, color='rgb(175,91,92)', strokeWidth=2)

alt.Chart(...)

In [1428]:
plot.fire.summary(mtbs, 'vlf', clim=(0, 0.05))

alt.HConcatChart(...)

In [1429]:
plot.fire.monthly(mtbs, 'vlf', clim=(0, 0.1))

alt.VConcatChart(...)

In [1433]:
fit_vars = ['tavg', 'ppt']

In [507]:
np.random.shuffle(mtbs['vlf'])

In [1434]:
model = fit.fire(x=climate[fit_vars], y=mtbs['vlf'], f=groups, crossval=False)

In [1435]:
prediction = model.predict(x=climate[fit_vars], f=groups)

In [1436]:
plot.fire.monthly(prediction, 'prob', clim=(0, 0.025))

alt.VConcatChart(...)

In [1449]:
plot.fire.evaluation(mtbs, prediction, 'vlf', 'prob', clim=(0, 0.02))

alt.HConcatChart(...)

In [1413]:
plot.fire.evaluation(mtbs, prediction, 'vlf', 'prob', clim=(0.005, 0.02))

alt.HConcatChart(...)

In [1439]:
x = mtbs['time']
y1 = mtbs['vlf'].mean(['x', 'y'])
y2 = prediction['prob'].mean(['x', 'y'])

(plot.line(x=x, y=y1, width=900, height=200, opacity=0.5, color='rgb(175,91,92)', strokeWidth=2)
+ plot.line(x=x, y=y2, width=900, height=200, color='rgb(175,91,92)', strokeWidth=2))

alt.LayerChart(...)

In [1440]:
print('correlation of annual trends')
np.corrcoef(
    mtbs['vlf'].groupby('time.year').mean().mean(['x', 'y']),
    prediction['prob'].groupby('time.year').mean().mean(['x', 'y'])
)[0,1]

correlation of annual trends


0.7704952520953599

In [1441]:
print('correlation of monthly trends')
np.corrcoef(
    mtbs['vlf'].groupby('time.month').mean().mean(['x', 'y']),
    prediction['prob'].groupby('time.month').mean().mean(['x', 'y'])
)[0,1]

correlation of monthly trends


0.9227232913428375

In [1442]:
print('correlation of spatial trends')
np.corrcoef(
    mtbs['vlf'].mean('time').values.flatten(),
    prediction['prob'].mean('time').values.flatten()
)[0,1]

correlation of spatial trends


0.45351626458092403

In [1446]:
model.train_roc

0.8791616999957409